# Analysis of the Impact of Weather on Traffic
## UMD CMSC320 Final Project by Thomas Wrona


In [12]:
import pandas as pd
import Geohash
import pupygrib as ppg
import numpy as np
import matplotlib.pyplot as plt
import os
import math
from itertools import product
import glob

In [14]:
stationdb = pd.read_csv("data/dot_traffic_stations_2015.txt").dropna(subset = ['latitude', 'longitude'])
stationdb.drop(stationdb.columns[[0,1,2,3,4,5,6,8,11,12,13,14,15,16,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,37,38,39,40,41,42,43,44,45,46,47,49,50,51,52,53,54]], inplace = True, axis = 1)
stationdb = stationdb.drop(stationdb[stationdb['latitude'] > 90].index)
stationdb = stationdb.drop(stationdb[stationdb['longitude'] > 180].index)
stationdb = stationdb.drop(stationdb[stationdb['longitude'] < 60].index)
stationdb = stationdb.drop(stationdb[stationdb['latitude'] == 0].index)
stationdb = stationdb.drop(stationdb[stationdb['longitude'] == 0].index)
stationdb.loc[:,'longitude'] = -stationdb['longitude']
stationdb.head()

,direction_of_travel,fips_county_code,fips_state_code,latitude,longitude,previous_station_id,station_id
0,7,59,6,33.850898,-117.814391,NaN,129130
1,5,77,6,37.874697,-121.219590,NaN,100190
2,1,93,6,41.441777,-122.435010,NaN,022940
3,5,35,49,40.516500,-111.891520,NaN,000302
4,7,27,34,40.892373,-74.484206,NaN,W01136


In [16]:
def finder(row):
    return os.path.exists(f'data/noaa-global-surface-summary-of-the-day/gsod_all_years/{row["maslib"]}-{row["wban"]}-2015.op')

iddb = pd.read_csv("data/master-location-identifier-database-20190717_release_public.csv", encoding = "ISO-8859-1", header = 4, low_memory = False).dropna(subset = ['maslib', 'lat_prp', 'lon_prp'])
iddb.loc[:,'maslib'] = iddb['maslib'].astype(int).astype(str).str.zfill(6)
iddb.fillna(value = {'wban':99999}, inplace = True)
iddb.loc[:,'wban'] = iddb['wban'].astype(int).astype(str).str.zfill(5)
iddb.loc[:,'file_exists'] = iddb.apply(finder, axis = 1)
usiddb = iddb[iddb['file_exists']]
usiddb.drop(usiddb.columns[[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,20,23]], inplace = True, axis = 1)
usiddb.head()

C:\Users\Thomas Wrona\Downloads\cmsc320\WPy64-3740\python-3.7.4.amd64\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,maslib,wban,lat_prp,lon_prp
4,409110,99999,36.706944,67.209444
64,409380,99999,34.210000,62.228333
188,663100,99999,-12.610000,13.403611
195,663250,99999,-12.383333,16.950000
197,661040,99999,-5.586667,12.188611


In [17]:
geo = '0123456789bcdefghjkmnpqrstuvwxyz'
geomap = {}
for i in range(len(geo)):
    geomap[geo[i]] = i

def base32toint(b32):
    totlen = len(b32)
    total = 0
    for i in range(len(b32)):
        total = total + geomap[b32[i]] * 32 ** (totlen - i - 1)
    return total

def geohasher(row, latcol, loncol):
    return base32toint(Geohash.encode(row[latcol], row[loncol], precision = 5))
    
stationdb.loc[:,'geohash'] = stationdb.apply(lambda row: geohasher(row, 'latitude', 'longitude'), axis = 1)
usiddb.loc[:,'geohash'] = usiddb.apply(lambda row: geohasher(row, 'lat_prp', 'lon_prp'), axis = 1)

stationdb = stationdb.sort_values(by = 'geohash')
usiddb = usiddb.sort_values(by = 'geohash')

combineddf = pd.merge_asof(left = stationdb, right = usiddb, on = 'geohash', direction = 'nearest')
combineddf.loc[:,'dist_lat'] = (combineddf['latitude'] - combineddf['lat_prp']) * 69
combineddf.loc[:,'dist_long'] = (combineddf['longitude'] - combineddf['lon_prp'])*69*np.cos(combineddf['latitude']*math.pi/180)
combineddf.loc[:,'dist'] = np.sqrt(np.square(combineddf['dist_lat']) + np.square(combineddf['dist_long']))
combineddf.drop(combineddf[combineddf['dist'] > 60].index, inplace = True)
combineddf.drop_duplicates(inplace = True)
combineddf.loc[:,'wban']= combineddf['wban'].astype(int)
combineddf.loc[:,'maslib']= combineddf['maslib'].astype(int)
combineddf.head()

C:\Users\Thomas Wrona\Downloads\cmsc320\WPy64-3740\python-3.7.4.amd64\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Thomas Wrona\Downloads\cmsc320\WPy64-3740\python-3.7.4.amd64\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,direction_of_travel,fips_county_code,fips_state_code,latitude,longitude,previous_station_id,station_id,geohash,maslib,wban,lat_prp,lon_prp,dist_lat,dist_long,dist
0,6,7,15,21.934802,-159.483514,NaN,050223,8649486,911650,22536,21.975983,-159.338958,-2.841489,-9.252315,9.678812
1,2,7,15,21.934802,-159.483514,NaN,050223,8649486,911650,22536,21.975983,-159.338958,-2.841489,-9.252315,9.678812
2,6,7,15,21.968091,-159.393066,NaN,050015,8649498,911650,22536,21.975983,-159.338958,-0.544548,-3.462375,3.504935
4,2,7,15,21.968091,-159.393066,NaN,050015,8649498,911650,22536,21.975983,-159.338958,-0.544548,-3.462375,3.504935
6,1,7,15,21.966216,-159.357469,NaN,051007,8649499,911650,22536,21.975983,-159.338958,-0.673923,-1.184536,1.362827


In [18]:
trafficdb = pd.read_csv("data/dot_traffic_2015.txt")
trafficdb.drop(trafficdb.columns[[1,4,6,7,8,9,10,11,-1]], inplace = True, axis = 1)
trafficdb = trafficdb.groupby(['station_id','fips_state_code','date', 'day_of_week', 'direction_of_travel'], as_index = False).sum()
trafficdb.head()

,station_id,fips_state_code,date,day_of_week,direction_of_travel,traffic_volume_counted_after_0000_to_0100,traffic_volume_counted_after_0100_to_0200,traffic_volume_counted_after_0200_to_0300,traffic_volume_counted_after_0300_to_0400,traffic_volume_counted_after_0400_to_0500,...,traffic_volume_counted_after_1400_to_1500,traffic_volume_counted_after_1500_to_1600,traffic_volume_counted_after_1600_to_1700,traffic_volume_counted_after_1700_to_1800,traffic_volume_counted_after_1800_to_1900,traffic_volume_counted_after_1900_to_2000,traffic_volume_counted_after_2000_to_2100,traffic_volume_counted_after_2100_to_2200,traffic_volume_counted_after_2200_to_2300,traffic_volume_counted_after_2300_to_2400
0,000001,8,2015-01-01,5,3,21,14,14,0,12,...,83,76,60,62,58,56,38,42,51,20
1,000001,8,2015-01-01,5,7,14,20,4,4,9,...,70,59,51,44,46,21,30,26,26,15
2,000001,8,2015-01-02,6,3,20,16,12,5,18,...,146,175,134,117,73,67,91,68,40,49
3,000001,8,2015-01-02,6,7,6,24,15,6,10,...,131,145,148,140,68,48,39,40,30,16
4,000001,8,2015-01-03,7,3,38,16,18,8,16,...,58,40,62,46,26,34,21,10,10,8


In [19]:
combineddf2 = pd.merge(trafficdb, combineddf, on = ['station_id','fips_state_code','direction_of_travel'])
combineddf2.loc[:,'date'] = combineddf2['date'].str.replace('-','').astype(int)
combineddf2.head()

,station_id,fips_state_code,date,day_of_week,direction_of_travel,traffic_volume_counted_after_0000_to_0100,traffic_volume_counted_after_0100_to_0200,traffic_volume_counted_after_0200_to_0300,traffic_volume_counted_after_0300_to_0400,traffic_volume_counted_after_0400_to_0500,...,longitude,previous_station_id,geohash,maslib,wban,lat_prp,lon_prp,dist_lat,dist_long,dist
0,000001,8,20150101,5,3,21,14,14,0,12,...,-102.572141,NaN,10409633,725626,24091,40.510325,-101.620147,8.237565,-49.852526,50.528525
1,000001,8,20150102,6,3,20,16,12,5,18,...,-102.572141,NaN,10409633,725626,24091,40.510325,-101.620147,8.237565,-49.852526,50.528525
2,000001,8,20150103,7,3,38,16,18,8,16,...,-102.572141,NaN,10409633,725626,24091,40.510325,-101.620147,8.237565,-49.852526,50.528525
3,000001,8,20150104,1,3,7,2,4,4,0,...,-102.572141,NaN,10409633,725626,24091,40.510325,-101.620147,8.237565,-49.852526,50.528525
4,000001,8,20150105,2,3,0,6,4,0,10,...,-102.572141,NaN,10409633,725626,24091,40.510325,-101.620147,8.237565,-49.852526,50.528525


In [20]:
weatherdb = pd.concat([pd.read_csv(f, delim_whitespace = True, skiprows = 1, names = ['STN', 'WBAN','YEARMODA','TEMP','TEMPCount','DEWP','DEWPCount','SLP','SLPCount','STP','STPCount','VISIB','VISIBCount','WDSP','WDSPCount','MXSPD','GUST','MAX','MIN','PRCP','SNDP','FRSHTT']) for f in glob.glob('data/noaa-global-surface-summary-of-the-day/gsod_all_years/*')],axis=0).reset_index()
weatherdb.head()

,index,STN,WBAN,YEARMODA,TEMP,TEMPCount,DEWP,DEWPCount,SLP,SLPCount,...,VISIBCount,WDSP,WDSPCount,MXSPD,GUST,MAX,MIN,PRCP,SNDP,FRSHTT
0,0,7070,99999,20150925,87.8,5,40.3,5,9999.9,0,...,0,0.2,5,1.0,999.9,93.2*,84.2*,0.00I,999.9,0
1,1,7070,99999,20150926,62.3,14,41.8,14,9999.9,0,...,0,0.0,14,1.0,999.9,86.0*,50.0*,0.00I,999.9,0
2,0,10010,99999,20150101,21.1,8,17.6,8,988.9,8,...,6,18.9,8,33.0,999.9,22.3,19.6,0.12G,999.9,1000
3,1,10010,99999,20150102,21.8,8,17.8,8,988.6,8,...,6,20.2,8,25.3,999.9,24.3,19.2,0.12E,999.9,1000
4,2,10010,99999,20150103,21.1,8,15.5,8,997.8,8,...,6,22.3,8,29.1,999.9,22.3,19.9*,0.01F,999.9,0


In [21]:
data = pd.merge(combineddf2, weatherdb, left_on = ['maslib','wban', 'date'], right_on = ['STN','WBAN', 'YEARMODA'], how = 'left')

In [22]:
data

,station_id,fips_state_code,date,day_of_week,direction_of_travel,traffic_volume_counted_after_0000_to_0100,traffic_volume_counted_after_0100_to_0200,traffic_volume_counted_after_0200_to_0300,traffic_volume_counted_after_0300_to_0400,traffic_volume_counted_after_0400_to_0500,...,VISIBCount,WDSP,WDSPCount,MXSPD,GUST,MAX,MIN,PRCP,SNDP,FRSHTT
0,000001,8,20150101,5,3,21,14,14,0,12,...,24.0,7.6,24.0,12.0,999.9,25.0*,0.0*,0.00G,999.9,0.0
1,000001,8,20150102,6,3,20,16,12,5,18,...,24.0,4.2,24.0,8.0,999.9,30.0,0.0,0.00G,999.9,0.0
2,000001,8,20150103,7,3,38,16,18,8,16,...,24.0,12.3,24.0,33.0,40.0,32.0,3.9,0.00G,999.9,101000.0
3,000001,8,20150104,1,3,7,2,4,4,0,...,24.0,10.4,24.0,27.0,35.0,12.0*,-6.0*,0.01G,999.9,1000.0
4,000001,8,20150105,2,3,0,6,4,0,10,...,24.0,6.5,24.0,14.0,999.9,24.1,-7.1,0.00G,999.9,0.0
5,000001,8,20150106,3,3,4,4,4,6,10,...,24.0,6.6,24.0,12.0,999.9,39.0*,12.2*,0.00G,999.9,0.0
6,000001,8,20150107,4,3,10,2,6,2,10,...,24.0,12.8,24.0,26.0,35.9,39.9,10.0,0.00G,999.9,0.0
7,000001,8,20150108,5,3,6,0,0,2,8,...,24.0,13.1,24.0,27.0,40.0,45.0,9.0,0.00G,999.9,0.0
8,000001,8,20150109,6,3,9,8,4,6,11,...,24.0,11.1,24.0,27.0,36.9,27.0*,5.0*,0.00G,999.9,1000.0
9,000001,8,20150110,7,3,9,13,0,2,8,...,24.0,6.8,24.0,12.0,999.9,45.0,3.9,0.00G,999.9,0.0
